# first attempt - simple reverse prediction

In [46]:
import numpy as np
import pandas as pd
from functions import *
from read_from_file_df import *
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [47]:
SIZE = 4
AMOUNT_BOARDS = 1000
AMOUNT_MOVES = 100
NUM_DICT = 1

In [48]:
gen = 2
name_df = f'{PATH_DF}\\{SIZE}-{AMOUNT_BOARDS}\\{SIZE}size_{AMOUNT_BOARDS}boards_{gen}gen_reverse'
reverse_df = pd.read_pickle(f'{name_df}.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\GameOfLifeFiles\\df\\\\4-1000\\4size_1000boards_2gen_reverse.pkl'

In [ ]:
print(name_df)

C:\GameOfLifeFiles\df\\4-100\\4size_100boards_2gen_reverse


In [ ]:
new_columns = [f'Col_{i}' for i in range(1, gen*SIZE*SIZE + 1)]
reverse_df_sort = reverse_df.sort_values(by = new_columns).reset_index(drop=True)
for i in reverse_df_sort.columns:
    reverse_df_sort[i] = reverse_df_sort[i].astype(int)

In [ ]:
print("reverse df:", len(reverse_df))
print("reverse df sort:",len(reverse_df_sort))

reverse df: 68
reverse df sort: 68


In [ ]:
# Step 1: Prepare Data
amount_features = len(reverse_df_sort.columns) - SIZE*SIZE #the previous boards
features = reverse_df_sort.iloc[:, :amount_features]
name_col = 'Col_' + str(amount_features + 1)  # Target: the first pixel in the board
target = reverse_df_sort[name_col]

# Step 2: Split Data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=365)

print("len x train: ", len(X_train))
print("len x test: ",len(X_test))
print("len y train: ",len(y_train))
print("len y test: ",len(y_test))

dt = tree.DecisionTreeClassifier(random_state=42)
dt = dt.fit(X_train, y_train)

y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

train_test_full_error = pd.concat([measure_error(y_train, y_train_pred, 'train'),
                                measure_error(y_test, y_test_pred, 'test')],
                                axis=1)

print(dt.tree_.node_count, dt.tree_.max_depth)
train_test_full_error

len x train:  61
len x test:  7
len y train:  61
len y test:  7
43 8


,train,test
accuracy,0.983607,0.714286
precision,1.000000,0.500000
recall,0.941176,0.500000
f1,0.969697,0.500000


In [ ]:
reverse_df_sort.iloc[17].values

array([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 1])

In [ ]:
features.iloc[17].values

array([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0])

In [ ]:
target.iloc[17]

0

In [ ]:
depth_to_check = [4,5,6,7,8,9,10,15,20,30,50,100]

for i in depth_to_check:
    dt, node_count, max_depth, train_test_full_error  = dec_tree_df(X_train,y_train, X_test, y_test, i, 42)
    
    name_file = f"{PATH_MODELS}\\reverse_model1\\{SIZE}\\decision_tree_size{SIZE}_reverse_df_model_{str(i)}.pkl"
    with open(name_file, 'wb') as file:
        pickle.dump(dt, file)
    
    print(node_count, max_depth)
    print(train_test_full_error)

17 4
              train      test
accuracy   0.819672  0.714286
precision  0.714286  0.500000
recall     0.588235  0.500000
f1         0.645161  0.500000


27 5
              train      test
accuracy   0.885246  0.714286
precision  1.000000  0.500000
recall     0.588235  0.500000
f1         0.740741  0.500000
35 6
              train      test
accuracy   0.950820  0.714286
precision  1.000000  0.500000
recall     0.823529  0.500000
f1         0.903226  0.500000
41 7
              train      test
accuracy   0.967213  0.714286
precision  1.000000  0.500000
recall     0.882353  0.500000
f1         0.937500  0.500000
43 8
              train      test
accuracy   0.983607  0.714286
precision  1.000000  0.500000
recall     0.941176  0.500000
f1         0.969697  0.500000
43 8
              train      test
accuracy   0.983607  0.714286
precision  1.000000  0.500000
recall     0.941176  0.500000
f1         0.969697  0.500000
43 8
              train      test
accuracy   0.983607  0.714286
precision  1.000000  0.500000
recall     0.941176  0.500000
f1         0.969697  0.500000
43 8
              train      test
accuracy   0.983607  0.714286
preci

In [ ]:
amount_features = len(reverse_df_sort.columns) - SIZE*SIZE #the previous boards
features = reverse_df_sort.iloc[:, :amount_features]
for i in range(SIZE*SIZE): # to any pixel in the expected board
    name_col = 'Col_' + str(i+amount_features + 1)
    target = reverse_df_sort[name_col]
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=613)
    dt, node_count, max_depth, train_test_full_error  = dec_tree_df(X_train,y_train, X_test, y_test, 10, 42)
    name_file = f"{PATH_MODELS}\\reverse1\\{SIZE}\\dt_size{SIZE}_r1_pixel_{str(i+1)}.pkl"
    with open(name_file, 'wb') as file:
        pickle.dump(dt, file)
    print(i)
    print(node_count, max_depth)
    print(train_test_full_error)

0
49 7
              train      test
accuracy   0.983607  0.714286
precision  1.000000  0.000000
recall     0.947368  0.000000
f1         0.972973  0.000000
1
45 9
              train      test
accuracy   0.983607  0.714286
precision  1.000000  1.000000
recall     0.950000  0.500000
f1         0.974359  0.666667
2
47 8
           train      test
accuracy     1.0  0.714286
precision    1.0  0.000000
recall       1.0  0.000000
f1           1.0  0.000000
3
35 8
           train      test
accuracy     1.0  0.571429
precision    1.0  0.000000
recall       1.0  0.000000
f1           1.0  0.000000
4
39 9
           train      test
accuracy     1.0  0.714286
precision    1.0  0.000000
recall       1.0  0.000000
f1           1.0  0.000000
5
41 8
           train      test
accuracy     1.0  0.428571
precision    1.0  0.000000
recall       1.0  0.000000
f1           1.0  0.000000
6
51 10
           train      test
accuracy     1.0  0.714286
precision    1.0  0.000000
recall       1.0  0.000000
f1

c:\Users\דרור\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\דרור\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\דרור\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(re

11
35 8
           train      test
accuracy     1.0  0.285714
precision    1.0  0.000000
recall       1.0  0.000000
f1           1.0  0.000000
12
41 9
           train      test
accuracy     1.0  0.571429
precision    1.0  1.000000
recall       1.0  0.250000
f1           1.0  0.400000
13
51 8
              train      test
accuracy   0.983607  0.714286
precision  1.000000  0.600000
recall     0.954545  1.000000
f1         0.976744  0.750000
14
55 10
              train      test
accuracy   0.983607  0.714286
precision  1.000000  0.000000
recall     0.947368  0.000000
f1         0.972973  0.000000
15
51 10
              train      test
accuracy   0.983607  0.571429
precision  1.000000  0.500000
recall     0.952381  0.333333
f1         0.975610  0.400000


c:\Users\דרור\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
X_test

,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,Col_10,Col_11,Col_12,Col_13,Col_14,Col_15,Col_16
45,0,1,1,0,0,0,0,0,0,1,1,0,0,1,0,0
15,0,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0
9,0,0,0,0,0,0,0,0,1,0,1,1,1,0,1,0
18,0,0,0,0,1,0,1,1,0,0,0,1,0,0,0,1
8,0,0,0,0,0,0,0,0,1,0,1,0,1,1,1,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
53,1,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0


In [ ]:
PATH_DATA_TEST = 'C:\\GameOfLifeFiles\\dataTest\\'

In [ ]:
name_file = f"{PATH_DATA_TEST}\\data_test_size_{SIZE}.pkl"
with open(name_file, 'wb') as file:
    pickle.dump(X_test, file)